<a href="https://colab.research.google.com/github/darshselarka1497/TSLA-Stock-Price-Predictor/blob/master/predictor_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# TSLA Price Prediction using LSTM using the closing prices in 2020 so far...

In [1]:
import math
import pandas_datareader as web
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal
Using TensorFlow backend.


In [ ]:
df = web.DataReader('TSLA', data_source='yahoo', start='2020-01-01', end='2020-07-22')
df

In [7]:
df.shape

(140, 6)

In [ ]:
#visualizing closing price for TSLA in 2020 (so far...)
plt.figure(figsize=(16,8))
plt.title('TSLA Close Price History')
plt.plot(df['Close'])
plt.xlabel("Date", fontsize=15)
plt.ylabel('USD', fontsize=15)
plt.show()